# NYC data preprocessing

In [5]:
# Importing all necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kstest
import numpy as np
from scipy import stats
from sklearn.preprocessing import PowerTransformer
from scipy.stats import rankdata, norm
pd.set_option('display.max_columns', 50)

In [ ]:
# Take a look at New York City airbnb dataset
nyc_df = pd.read_csv('./NYC_airbnb.csv')
nyc_df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,1189243425411300671,The Grand III,61391963,Stay With Vibe,Manhattan,Murray Hill,40.750251,-73.977745,Entire home/apt,58.0,30,1,2024-08-02,0.86,130,357,1,NaN
1,651593916026998398,Cheerful one bedroom townhouse with backyard,136812643,Marah,Brooklyn,Flatlands,40.624760,-73.935560,Private room,80.0,30,0,NaN,NaN,1,365,0,NaN
2,310325,Large Sunny Bedroom with Bay Window,745069,Kimberly,Manhattan,Harlem,40.823590,-73.946010,Private room,75.0,30,31,2019-09-28,0.21,2,335,0,NaN
3,572612125615500056,Room by Sunny & Bay! Sunset Park & Bay Ridge,358089614,Joshua,Brooklyn,Sunset Park,40.639520,-74.014840,Private room,45.0,30,6,2024-07-30,0.21,2,336,1,NaN
4,1020282701018874374,Trendy Private Bedroom,483056418,Kristina,Brooklyn,Bedford-Stuyvesant,40.678946,-73.943759,Private room,47.0,30,0,NaN,NaN,24,269,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37536,942866549062627160,"Small Cozy Bedroom in Queens, NY",345043452,Chelsea,Queens,St. Albans,40.698570,-73.758480,Private room,80.0,3,15,2024-08-31,3.44,1,23,15,OSE-STRREG-0000376
37537,16232783,Private rooms (shared spaces with me in the unit),147708401,Ирина,Staten Island,Great Kills,40.556210,-74.158050,Private room,68.0,30,58,2024-05-31,0.63,1,154,4,NaN
37538,719062366977533285,Modern Beach Getaway,480094583,William,Queens,Belle Harbor,40.576920,-73.845240,Entire home/apt,150.0,30,26,2023-09-04,1.14,1,177,0,NaN
37539,36639976,Cozy room in sunnyside close to all,275402693,Shelbie,Queens,Sunnyside,40.738980,-73.921570,Private room,NaN,30,25,2019-12-22,0.40,2,0,0,NaN


In [6]:
# Looking through the information about dataset (looking for empty values)
nyc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37541 entries, 0 to 37540
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              37541 non-null  int64  
 1   name                            37539 non-null  object 
 2   host_id                         37541 non-null  int64  
 3   host_name                       37536 non-null  object 
 4   neighbourhood_group             37541 non-null  object 
 5   neighbourhood                   37541 non-null  object 
 6   latitude                        37541 non-null  float64
 7   longitude                       37541 non-null  float64
 8   room_type                       37541 non-null  object 
 9   price                           22790 non-null  float64
 10  minimum_nights                  37541 non-null  int64  
 11  number_of_reviews               37541 non-null  int64  
 12  last_review                     

In [7]:
# We can drop license column because it has too many empty values
nyc_df.drop('license', axis=1, inplace=True)
nyc_df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,1189243425411300671,The Grand III,61391963,Stay With Vibe,Manhattan,Murray Hill,40.750251,-73.977745,Entire home/apt,58.0,30,1,2024-08-02,0.86,130,357,1
1,651593916026998398,Cheerful one bedroom townhouse with backyard,136812643,Marah,Brooklyn,Flatlands,40.624760,-73.935560,Private room,80.0,30,0,NaN,NaN,1,365,0
2,310325,Large Sunny Bedroom with Bay Window,745069,Kimberly,Manhattan,Harlem,40.823590,-73.946010,Private room,75.0,30,31,2019-09-28,0.21,2,335,0
3,572612125615500056,Room by Sunny & Bay! Sunset Park & Bay Ridge,358089614,Joshua,Brooklyn,Sunset Park,40.639520,-74.014840,Private room,45.0,30,6,2024-07-30,0.21,2,336,1
4,1020282701018874374,Trendy Private Bedroom,483056418,Kristina,Brooklyn,Bedford-Stuyvesant,40.678946,-73.943759,Private room,47.0,30,0,NaN,NaN,24,269,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37536,942866549062627160,"Small Cozy Bedroom in Queens, NY",345043452,Chelsea,Queens,St. Albans,40.698570,-73.758480,Private room,80.0,3,15,2024-08-31,3.44,1,23,15
37537,16232783,Private rooms (shared spaces with me in the unit),147708401,Ирина,Staten Island,Great Kills,40.556210,-74.158050,Private room,68.0,30,58,2024-05-31,0.63,1,154,4
37538,719062366977533285,Modern Beach Getaway,480094583,William,Queens,Belle Harbor,40.576920,-73.845240,Entire home/apt,150.0,30,26,2023-09-04,1.14,1,177,0
37539,36639976,Cozy room in sunnyside close to all,275402693,Shelbie,Queens,Sunnyside,40.738980,-73.921570,Private room,NaN,30,25,2019-12-22,0.40,2,0,0


In [10]:
#
names = nyc_df['name']
names_set = set(names)
print(len(names_set))

35865
